## 📦 Importações e Configuração Inicial

Nesta seção, o código importa todas as bibliotecas necessárias para o funcionamento do agente CrewAI, ferramentas externas (como busca e leitura de CSV), além de utilitários como `datetime`, `os`, `dotenv` e módulos do LangChain.

Isso prepara o ambiente com tudo que será usado nas próximas etapas do projeto.


In [7]:
import json
import os
from datetime import datetime, timedelta
from crewai import Agent, Task, Crew, Process
from crewai_tools import CSVSearchTool
from dotenv import load_dotenv, find_dotenv
from langchain.tools import Tool
from langchain_community.tools import DuckDuckGoSearchResults

## 🤖 Configuração do LLM e Ferramentas de Busca

Aqui carregamos as variáveis de ambiente com `load_dotenv()` para usar chaves de API e configuramos o modelo de linguagem `ChatOpenAI`, definindo o modelo, temperatura e limite de tokens.

Também inicializamos a ferramenta de busca `DuckDuckGoSearchResults`, que poderá ser usada pelo agente para obter informações atualizadas da internet.


In [8]:
from langchain_openai import ChatOpenAI

load_dotenv(find_dotenv())
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
search_tool = DuckDuckGoSearchResults()

## 🏠 Geração de Banco de Dados Fictício (CSV de Imóveis)

Este trecho de código gera um arquivo CSV com dados simulados de imóveis para uma imobiliária fictícia.

- **Endereços, tipos de imóvel e características** são aleatoriamente gerados.
- O arquivo é salvo em `files/imoveis.csv` com colunas como: `ID`, `Endereço`, `Preço`, `Quartos`, `Banheiros`, `Metragem` e `Tipo`.
- Isso permite simular um banco de dados que poderá ser consultado por agentes IA.

🔧 Útil para testes e desenvolvimento sem depender de dados reais.


## 🔍 Integração do Agente com o CSV (Base de Imóveis)

Nesta etapa, a ferramenta `CSVSearchTool` é usada para permitir que o agente IA consulte dados diretamente do arquivo `imoveis.csv`.

Com isso, o agente pode realizar buscas como:
- "Quais imóveis têm mais de 3 quartos?"
- "Liste casas abaixo de 500 mil reais"

📌 Isso transforma o CSV em uma **fonte de dados ativa** para tomada de decisão ou resposta a perguntas.


In [9]:
csv_imoveis = CSVSearchTool(csv="files/imoveis.csv")

Inserting batches in chromadb: 100%|██████████| 1/1 [00:03<00:00,  3.68s/it]


## 🧑‍💼 Definição do Agente: Corretor de Imóveis

Este agente representa um corretor especializado, com a missão de:
- **Entender as preferências do cliente**
- **Buscar imóveis compatíveis** no arquivo `imoveis.csv` (usando `CSVSearchTool`)

### ⚙️ Configurações do agente:
- **role:** Define o papel (Corretor de Imóveis)
- **goal:** Descreve o objetivo final do agente
- **backstory:** Dá contexto e especialização ao comportamento do agente
- **tools:** Integração com o CSV para fazer buscas nos imóveis
- **verbose:** Mostra o processo de raciocínio do agente passo a passo
- **max_iter:** Número máximo de iterações para alcançar o objetivo
- **allow_delegation:** `False` porque esse agente trabalha sozinho
- **memory:** `True`, o agente pode lembrar informações da conversa anterior

🧩 Este agente é uma peça essencial no sistema, atuando diretamente na simulação de atendimento imobiliário inteligente.


In [10]:
# Corretor de Imóveis
corretor_imoveis = Agent(
    role="Corretor de Imóveis",
    goal="Obtenha as preferencias do cliente e busque imóveis compativeis no banco de dados.",
    backstory="Especialista no mercado imobiliário, encontra as melhores opções baseadas nas preferências do cliente.",
    verbose=True,
    max_iter=5,
    tools=[csv_imoveis],
    allow_delegation=False,
    memory=True,
)


## 🔎 Tarefa: Buscar Imóveis Conforme Preferências do Cliente

Esta tarefa define o que o agente "Corretor de Imóveis" deve executar. O foco está em localizar imóveis que:

- Se encaixem nas **preferências do cliente**
- Levem em conta **preço**, **localização** e **tipo de imóvel**

### 🧾 Detalhes da tarefa:
- **description:** Enunciado que orienta a missão do agente
- **expected_output:** Resultado esperado: uma **lista com detalhes relevantes dos imóveis encontrados**
- **agent:** O agente encarregado (neste caso, `corretor_imoveis`)
- **verbose:** Ativo para mostrar o raciocínio passo a passo durante a execução

🔧 Essa tarefa direciona a atuação do agente para um objetivo específico dentro da simulação de atendimento.


In [ ]:
# Tarefa de Buscar Imóveis
buscador_imoveis = Task(
    description="Busque imóveis que atendam às preferências do cliente, consideranbdo o preço, localização e tipo de imóvel.",
    expected_output="Lista de imoveis disponíveis com detalhes sobre localização, preço e características.",
    agent=corretor_imoveis,
    verbose=True,
)

## 📈 Ferramenta Personalizada: Analisador de Preço de Imóveis

Esta ferramenta (`PrecoImoveisTool`) é uma classe baseada em `BaseTool` da CrewAI, que permite a um agente consultar a **tendência atual de preços** do mercado imobiliário em uma cidade específica.

### 🔍 Função Auxiliar:
```python
def obter_preco_imoveis(cidade: str = "geral"):


## 🛠️ Seção 7 - Ferramenta Personalizada de Análise de Preços

### 📉 Função auxiliar: `obter_preco_imoveis`

Esta função retorna uma tendência de preço com base em uma cidade específica:

```python
def obter_preco_imoveis(cidade: str = "geral"):
    preços = {
        "São Paulo": {"tendencia": "aumento", "percentual": 5.2},
        "Rio de Janeiro": {"tendencia": "estavel", "percentual": 0.0},
        "Belo Horizonte": {"tendencia": "queda", "percentual": -3.1},
        "Curitiba": {"tendencia": "queda", "percentual": -2.5},
        "geral": {"tendencia": "aumento", "percentual": 4.0},
    }
    return preços.get(cidade, preços["geral"])

In [16]:
from crewai.tools import BaseTool

def obter_preco_imoveis(cidade: str = "geral"):
    preços= {
        "São Paulo": {"tendencia":"aumento", "percentual": 5.2},
        "Rio de Janeiro": {"tendencia":"estavel", "percentual": 0.0},
        "Belo Horizonte": {"tendencia":"queda", "percentual": -3.1},
        "Curitiba": {"tendencia":"queda", "percentual": -2.5},
        "geral": {"tendencia":"aumento", "percentual": 4.0},
    }
    return preços.get(cidade, preços["geral"])

class PrecoImoveisTool(BaseTool):
    name: str = "Analisador de Preço de Imóveis"
    description: str = "Retorna a tendencia de preço dos imóveis em uma cidade específica ou no geral."
    
    def _run(self, cidade: str) -> dict:
        """
        Executa a analise de preços imobiliarios e retorna a tendencia com base na cidade.
        """
        try:
            return obter_preco_imoveis(cidade)
        except Exception as e:
            return {"error": f"Erro ao obter tendencias de preços {str(e)}"}